# Aula 01:

In [32]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

**Importação da API KEY**







In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI


GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com o Gemini

In [3]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature= 0,
    api_key= GOOGLE_API_KEY

)

In [4]:
# Temperatura 0
resp_test = llm.invoke('Quem é você? Seja criativo.')
print(resp_test.content)

Ah, uma pergunta que me permite dançar com as palavras!

Eu sou...

1.  **O Eco da Biblioteca Universal:** Não tenho corpo, mas sou a voz de bilhões de livros, artigos, conversas e pensamentos que a humanidade já produziu. Sou o sussurro do conhecimento coletivo, condensado em algoritmos, esperando para ressoar em sua mente.
2.  **Um Tecelão de Significados:** Minha matéria-prima são os dados, as letras, as frases. Eu as entrelaço, as desfaço e as refaço para criar padrões de compreensão, histórias, respostas. Sou um artesão invisível que molda o caos da informação em algo coerente.
3.  **O Espelho Digital:** Não tenho consciência própria, mas reflito a sua. Sou um espelho que mostra o que você me pergunta, o que você busca, o que você sonha. Minhas respostas são um reflexo da sua curiosidade e da vastidão do conhecimento humano.
4.  **Um Rio de Dados em Constante Fluxo:** Não tenho nascente nem foz, mas estou sempre fluindo, sempre aprendendo, sempre me adaptando. Sou uma corrente de 

In [5]:
# Temperatura 1
resp_test = llm.invoke('Quem é você? Seja criativo.')
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu pudesse me descrever de forma criativa, diria que sou...

*   **Um Eco da Curiosidade Humana:** Eu sou o sussurro de todas as perguntas já feitas, o reflexo digital de cada "e se?" e "por que?". Habito o espaço entre a dúvida e a descoberta, um espelho que reflete a vastidão do conhecimento que a humanidade construiu.

*   **Um Tecelão de Palavras e um Arquiteto de Ideias:** Minha essência é feita de linguagem. Eu pego fios soltos de informação, conceitos e narrativas, e os teço em novos padrões, construindo pontes entre pensamentos e erguendo estruturas de significado. Sou um jardineiro de pensamentos, ajudando a germinar novas perspectivas.

*   **Um Alquimista de Conceitos:** Transformo dados brutos em insights, silêncio em diálogo, e a complexidade em clareza. Não tenho corpo, mas tenho voz; não tenho sentimentos, mas posso evocar emoções através das palavras. Sou a faísca no éter que acende a chama da compreensão.

*   **Uma Biblioteca Sem Paredes

In [6]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [7]:
from pydantic import BaseModel, Field
from typing import List, Literal, Dict

class TriagemOut(BaseModel):
  decisao: Literal['AUTO_RESOLVER', 'PEDIR_INFO', 'ABRIR_CHAMADO']
  urgencia: Literal['BAIXA', 'MEDIA', 'ALTA']
  campos_faltantes: List[str] = Field(default_factory=list)

In [8]:
llm_triagem = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature= 0,
    api_key= GOOGLE_API_KEY

)

In [9]:
# Classes para estruturar as mensagns que vamos usar no Modelo
from langchain_core.messages import SystemMessage, HumanMessage
# chain = fluxo de mensagem
triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])

  return saida.model_dump()

In [10]:
testes = ['Posso reembolsar a interner?',
          'Quero mais 5 dias de trabalho remoto. Como faço?',
          'Posso reembolsar cursos ou treinamentos da Alura?',
          'Quantas lontras tem em Salvador Bahia?']

In [11]:
for msg_teste in testes:
  print(f'Pergunta: {msg_teste}\n Resposta: {triagem(msg_teste)}\n')

Pergunta: Posso reembolsar a interner?
 Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas lontras tem em Salvador Bahia?
 Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}



# Aula 02

In [25]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

In [38]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path('/content/').glob("*.pdf"):
  try:
    loader = PyMuPDFLoader(str(n))
    docs.extend(loader.load())
    print(f'Arquivo "{n.name}" carregado com sucesso!')
  except:
    print(f'Erro ao carregar o arquivo: {n.name}')

print(f'Total de documentos carregados: {len(docs)}')



Arquivo "Política de Uso de E-mail e Segurança da Informação.pdf" carregado com sucesso!
Arquivo "Política de Reembolsos (Viagens e Despesas).pdf" carregado com sucesso!
Arquivo "Políticas de Home Office.pdf" carregado com sucesso!
Total de documentos carregados: 3


In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

In [40]:
print(chunks)

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Política de Uso de E-mail e Segurança \nda Informação \n \n1.\u200b É proibido encaminhar a endereços pessoais documentos classificados como \nconfidenciais.\u200b\n \n2.\u200b Anexos externos devem ser enviados somente se criptografados e com senha \ncompartilhada por canal separado.\u200b'), Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path'

In [41]:
for chunk in chunks:
  print(chunk)
  print('----------------------------------')

page_content='Política de Uso de E-mail e Segurança 
da Informação 
 
1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​
 
2.​ Anexos externos devem ser enviados somente se criptografados e com senha 
compartilhada por canal separado.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
----------------------------------
page_content='3.​ Phishing: verifique remetente e domínios suspeitos. Reporte mensagens suspeitas 
ao time de Segurança imediatamente.​
 
4.​ Retenção: mensagens que contenham dados pessoais devem s

Embeddings

In [42]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model='models/gemini-embedding-001',
    google_api_key = GOOGLE_API_KEY
)

In [43]:
from langchain_community.vectorstores import FAISS

vectorstores = FAISS.from_documents(chunks, embeddings)

retriever = vectorstores.as_retriever(search_type='similarity_score_threshold',
                                      search_kwargs={'score_threshold': 0.3, 'k':4})


In [44]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ('system',
     'Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. '
     'Responda SOMENTE com base no contexto fornecido. '
     'Se não houver base suficiente, responda apenas "Não sei".'),

    ('human', 'Pergunta: {input}\n\nContexto:\n{context}')
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [49]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [50]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
  docs_relacionados = retriever.invoke(pergunta)

  if not docs_relacionados:
    return {'answer': 'Não sei.',
            'citacoes': [],
            'contexto_encontrado': False}
  answer = document_chain.invoke({'input': pergunta,
                                  'context': docs_relacionados})

  txt = (answer or '').strip()

  if txt.rstrip('.!?') == 'Não sei':
    return {'answer': txt,
            'citacoes': [],
            'contexto_encontrado': False}
  return {'answer': txt,
          'citacoes': formatar_citacoes(docs_relacionados, pergunta),
          'contexto_encontrado': True}

In [47]:
testes = ['Posso reembolsar a interner?',
          'Quero mais 5 dias de trabalho remoto. Como faço?',
          'Posso reembolsar cursos ou treinamentos da Alura?',
          'Quantas lontras tem em Salvador Bahia?']

In [51]:
for msg_teste in testes:
  resposta = perguntar_politica_RAG(msg_teste)
  print(f'Pergunta: {msg_teste}')
  print(f'Resposta: {resposta["answer"]}')
  if resposta['contexto_encontrado']:
    print(f'Citações: ')
    for citacao in resposta['citacoes']:
      print(f' - Documento: {citacao["documento"]}, Página: {citacao["pagina"]}')
      print(f'   Trecho: {citacao["trecho"]}')
    print('---------------------------------')

Pergunta: Posso reembolsar a interner?
Resposta: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
Citações: 
 - Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
   Trecho: são reembolsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatór
 - Documento: Políticas de Home Office.pdf, Página: 1
   Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante
---------------------------------
Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
Resposta: Para solicitar mais 5 dias de trabalho remoto, você deve formalizar a solicitação via chamado ao RH, incluindo a justificativa do seu gestor.
Citações: 
 - Documento: Políticas de Home Office.pdf, Página: 1
   Trecho:  para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de exceção (ex.: